# Advanced Pydantic AI Patterns

This notebook explores pydantic-ai's more powerful features:

- **Structured Output** - Type-safe responses with Pydantic models
- **Dependency Injection** - Pass runtime context to tools
- **Dynamic System Prompts** - Generate prompts based on state
- **Tool Context** - Access dependencies and usage info in tools
- **Model Settings** - Control temperature, tokens, etc.
- **Usage Tracking** - Monitor token consumption

In [ ]:
from dotenv import load_dotenv

load_dotenv("../.env")

from dataclasses import dataclass

from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext

from pydantic_ai_jupyter import run_in_jupyter

## Structured Output

Instead of free-form text, agents can return validated Pydantic models. This ensures type safety and makes the output immediately usable in your code.

In [ ]:
class MovieRecommendation(BaseModel):
    """A movie recommendation with structured details."""

    title: str = Field(description="The movie title")
    year: int = Field(description="Year of release")
    genre: str = Field(description="Primary genre")
    reason: str = Field(description="Why this movie is recommended")
    rating: float = Field(ge=0, le=10, description="Expected enjoyment rating 0-10")


movie_agent = Agent(
    "openai:gpt-4o-mini",
    output_type=MovieRecommendation,
    system_prompt="You recommend movies based on user preferences. Always provide thoughtful recommendations.",
)

In [ ]:
result = await run_in_jupyter(
    movie_agent,
    "I love mind-bending sci-fi like Inception and The Matrix",
)

# The output is a validated Pydantic model
print(f"\nStructured output: {result.output}")
print(f"Title: {result.output.title}")
print(f"Rating: {result.output.rating}/10")

## Dependency Injection

Pass runtime context (database connections, user info, API clients) to your agent and tools using the `deps` parameter. This keeps your agent definition clean while allowing dynamic behavior.

In [ ]:
@dataclass
class UserContext:
    """Runtime context passed to the agent."""

    user_name: str
    user_tier: str  # "free" or "premium"
    preferences: dict[str, str]


# Define agent with dependency type
support_agent = Agent(
    "openai:gpt-4o-mini",
    deps_type=UserContext,
    system_prompt="You are a helpful support assistant.",
)


@support_agent.system_prompt
def add_user_context(ctx: RunContext[UserContext]) -> str:
    """Dynamic system prompt based on user context."""
    tier_info = (
        "This is a premium user - provide detailed, priority support."
        if ctx.deps.user_tier == "premium"
        else "This is a free tier user."
    )
    return f"User: {ctx.deps.user_name}. {tier_info}"


@support_agent.tool
def get_account_status(ctx: RunContext[UserContext]) -> str:
    """Get the user's account status and preferences."""
    return f"Account: {ctx.deps.user_name}, Tier: {ctx.deps.user_tier}, Preferences: {ctx.deps.preferences}"

In [ ]:
# Create user context
user = UserContext(
    user_name="Alice",
    user_tier="premium",
    preferences={"theme": "dark", "notifications": "email"},
)

result = await run_in_jupyter(
    support_agent,
    "What's my account status and can you help me update my preferences?",
    deps=user,
)

## Model Settings

Control model behavior with settings like temperature, max tokens, and more. You can set defaults on the agent or override per-run.

In [ ]:
creative_agent = Agent(
    "openai:gpt-4o-mini",
    system_prompt="You are a creative writing assistant.",
)

# High temperature for creative output
result = await run_in_jupyter(
    creative_agent,
    "Write a one-sentence story about a robot discovering emotions.",
    model_settings={"temperature": 0.9},
)

In [ ]:
# Low temperature for consistent, focused output
result = await run_in_jupyter(
    creative_agent,
    "Write a one-sentence story about a robot discovering emotions.",
    model_settings={"temperature": 0.1},
)

## Usage Tracking

Monitor token consumption and set limits. The result object contains detailed usage information.

In [ ]:
from pydantic_ai import UsageLimits

simple_agent = Agent("openai:gpt-4o-mini")

result = await run_in_jupyter(
    simple_agent,
    "Explain quantum computing in exactly two sentences.",
    usage_limits=UsageLimits(response_tokens_limit=100),
)

# Inspect usage - note: usage() is a method
usage = result.usage()
print("\nUsage Statistics:")
print(f"  Input tokens: {usage.input_tokens}")
print(f"  Output tokens: {usage.output_tokens}")
print(f"  Total: {usage.input_tokens + usage.output_tokens}")
print(f"  Requests: {usage.requests}")

## Putting It Together: A Data Analysis Agent

Here's a more complete example combining structured output, dependencies, and tools for a data analysis use case.

In [ ]:
import json


class AnalysisResult(BaseModel):
    """Structured analysis output."""

    summary: str = Field(description="Brief summary of findings")
    key_metrics: dict[str, float] = Field(description="Key numeric metrics identified")
    recommendations: list[str] = Field(description="Actionable recommendations")
    confidence: float = Field(ge=0, le=1, description="Confidence in the analysis")


@dataclass
class AnalysisContext:
    """Context for the analysis agent."""

    dataset_name: str
    data: dict  # Simulated data


analysis_agent = Agent(
    "openai:gpt-4o-mini",
    deps_type=AnalysisContext,
    output_type=AnalysisResult,
    system_prompt="You are a data analyst. Analyze data and provide actionable insights.",
)


@analysis_agent.tool
def query_data(ctx: RunContext[AnalysisContext], metric: str) -> str:
    """Query a specific metric from the dataset."""
    data = ctx.deps.data
    if metric in data:
        return json.dumps({metric: data[metric]})
    return f"Metric '{metric}' not found. Available: {list(data.keys())}"


@analysis_agent.tool
def get_dataset_info(ctx: RunContext[AnalysisContext]) -> str:
    """Get information about the current dataset."""
    return f"Dataset: {ctx.deps.dataset_name}, Metrics: {list(ctx.deps.data.keys())}"

In [ ]:
# Simulated sales data
sales_data = AnalysisContext(
    dataset_name="Q4 2024 Sales",
    data={
        "total_revenue": 1_250_000,
        "units_sold": 8_500,
        "avg_order_value": 147.06,
        "return_rate": 0.032,
        "customer_satisfaction": 4.2,
        "repeat_customer_rate": 0.45,
    },
)

result = await run_in_jupyter(
    analysis_agent,
    "Analyze our Q4 sales performance and suggest improvements for Q1.",
    deps=sales_data,
)

# Access the structured output
print(f"\n{'=' * 50}")
print(f"Analysis Summary: {result.output.summary}")
print(f"Confidence: {result.output.confidence:.0%}")
print("\nRecommendations:")
for i, rec in enumerate(result.output.recommendations, 1):
    print(f"  {i}. {rec}")

## Runtime Model Selection

You can override the model at runtime, useful for testing or cost optimization.

In [ ]:
# Agent defined with one model...
flexible_agent = Agent(
    "openai:gpt-4o-mini",
    system_prompt="You are a helpful assistant.",
)

# ...but you can override at runtime
result = await run_in_jupyter(
    flexible_agent,
    "What's 2 + 2?",
    model="openai:gpt-4o",  # Upgrade to gpt-4o for this run
)

## Summary

Key pydantic-ai patterns covered:

| Pattern | Use Case |
|---------|----------|
| `output_type=Model` | Type-safe structured responses |
| `deps_type` + `RunContext` | Dependency injection for tools |
| `@agent.system_prompt` | Dynamic prompts based on context |
| `model_settings={}` | Control temperature, tokens, etc. |
| `UsageLimits` | Monitor and limit token usage |
| `model="..."` | Override model at runtime |

For more details, see the [pydantic-ai documentation](https://ai.pydantic.dev/).